# Example Seldon Core Deployments using Helm
<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequistes
You will need
 - [Git clone of Seldon Core](https://github.com/SeldonIO/seldon-core)
 - A running Kubernetes cluster with kubectl authenticated
 - [python grpc tools](https://grpc.io/docs/quickstart/python.html)
 - [Helm client](https://helm.sh/)

### Creating a Kubernetes Cluster

Follow the [Kubernetes documentation to create a cluster](https://kubernetes.io/docs/setup/).

Once created ensure ```kubectl``` is authenticated against the running cluster.

# Setup

In [1]:
!kubectl create namespace seldon

namespace/seldon created


In [1]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "minikube" modified.


In [3]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


# Install Helm

In [4]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount/tiller created
clusterrolebinding.rbac.authorization.k8s.io/tiller created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [5]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


## Start seldon-core

In [7]:
!helm install ../../helm-charts/seldon-core-controller --name seldon-core --set usage_metrics.enabled=true --namespace seldon-system

NAME:   seldon-core
LAST DEPLOYED: Thu May  9 10:34:26 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                                   TYPE    DATA  AGE
seldon-operator-webhook-server-secret  Opaque  0     0s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1/ClusterRole
seldon-operator-manager-role  0s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  0s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.106.183.171  <none>       443/TCP  0s

==> v1/StatefulSet
NAME                                DESIRED  CURRENT  AGE
seldon-operator-controller-manager  1        1        0s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
seldon-operator-cont

In [10]:
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

partitioned roll out complete: 1 new pods have been updated...


## Setup Ingress
There are gRPC issues with the latest Ambassador, so we rewcommend 0.40.2 until these are fixed.

In [11]:
!helm install stable/ambassador --name ambassador --set image.tag=0.40.2

Error: a release named ambassador already exists.
Run: helm ls --all ambassador; to check the status of the release
Or run: helm del --purge ambassador; to delete it


In [12]:
!kubectl rollout status deployment.apps/ambassador

deployment "ambassador" successfully rolled out


In [3]:
! helm install ../../helm-charts/seldon-core-oauth-gateway --name seldon-gateway --namespace seldon

NAME:   seldon-gateway
LAST DEPLOYED: Thu May  9 13:32:34 2019
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRoleBinding
NAME           AGE
seldon         0s
seldon-seldon  0s

==> v1/Service
NAME                             TYPE       CLUSTER-IP      EXTERNAL-IP  PORT(S)                        AGE
seldon-gateway-redis-master      ClusterIP  10.105.144.238  <none>       6379/TCP                       0s
seldon-gateway-seldon-apiserver  NodePort   10.111.157.247  <none>       8080:30110/TCP,5000:30172/TCP  0s

==> v1beta1/Deployment
NAME                             DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-gateway-seldon-apiserver  1        1        1           0          0s

==> v1beta2/StatefulSet
NAME                         DESIRED  CURRENT  AGE
seldon-gateway-redis-master  1        1        0s

==> v1/Pod(related)
NAME                                              READY  STATUS             RESTARTS  AGE
seldon-gateway-seldon-apiserver-85db7687b9-xzn7t  0/1  

In [5]:
!kubectl rollout status deployment.apps/seldon-gateway-seldon-apiserver

deployment "seldon-gateway-seldon-apiserver" successfully rolled out


## Set up REST and gRPC methods

**Ensure you port forward ambassador and API gateway**:

```
kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8002:8080
```

```
kubectl port-forward $(kubectl get pods -n seldon -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].metadata.name}') -n seldon 8004:5000
```

## Serve Single Model

In [16]:
!kubectl create namespace test1

namespace/test1 created


In [17]:
!helm install ../../helm-charts/seldon-single-model --name mymodel --set oauth.key=oauth-key --set oauth.secret=oauth-secret --namespace test1

NAME:   mymodel
LAST DEPLOYED: Thu May  9 11:10:13 2019
NAMESPACE: test1
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME     AGE
mymodel  0s




In [18]:
!kubectl rollout status deploy/mymodel-mymodel-7cd068f -n test1

Waiting for deployment "mymodel-mymodel-7cd068f" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymodel-mymodel-7cd068f" successfully rolled out


### Get predictions

In [6]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mymodel",namespace="test1",oauth_key="oauth-key",oauth_secret="oauth-secret")

#### REST Request

In [7]:
p = sc.predict(gateway="seldon",transport="rest")
print(p)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.48380379505104043
  }
}

Response:
meta {
  puid: "1p875752kj5uh12b8h78ls3ueu"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.08070082081279971
  }
}



In [8]:
p = sc.predict(gateway="ambassador",transport="rest")
print(p)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.4366478560418421
  }
}

Response:
meta {
  puid: "ecc6f4vqarkgjdr6lhvrn8kuh4"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07727086092703452
  }
}



#### gRPC Request

In [9]:
p = sc.predict(gateway="ambassador",transport="grpc")
print(p)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.7613856135572777
  }
}

Response:
meta {
  puid: "t2ahkipoe6fpi7omipb50psler"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.10383875884578245
  }
}



In [10]:
p = sc.predict(gateway="seldon",transport="grpc")
print(p)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.41050638142891616
  }
}

Response:
meta {
  puid: "8e81ftst2re3988t91t993tlg9"
  requestPath {
    key: "classifier"
    value: "seldonio/mock_classifier:1.0"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07542744792497805
  }
}



In [11]:
!helm delete mymodel --purge

release "mymodel" deleted


## Serve AB Test

In [12]:
!helm install ../../helm-charts/seldon-abtest --name myabtest --set oauth.key=oauth-key --set oauth.secret=oauth-secret --namespace test1

NAME:   myabtest
LAST DEPLOYED: Thu May  9 14:10:03 2019
NAMESPACE: test1
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME      AGE
myabtest  1s




In [13]:
!kubectl rollout status deploy/myabtest-abtest-41de5b8 -n test1
!kubectl rollout status deploy/myabtest-abtest-df66c5c -n test1

Waiting for deployment "myabtest-abtest-41de5b8" rollout to finish: 0 of 1 updated replicas are available...
deployment "myabtest-abtest-41de5b8" successfully rolled out
deployment "myabtest-abtest-df66c5c" successfully rolled out


### Get predictions

In [14]:
sc = SeldonClient(deployment_name="myabtest",namespace="test1",oauth_key="oauth-key",oauth_secret="oauth-secret")

In [15]:
r = sc.predict(gateway="seldon",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.7708073955451095
  }
}

Response:
meta {
  puid: "vssct0pmpvb8jn7m0tr5qqat2j"
  routing {
    key: "myabtest"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
    value: ""
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.10471879300072064
  }
}



In [16]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.19907312279876477
  }
}

Response:
meta {
  puid: "vt20gbuqvmnp6ut7mtuni85mj9"
  routing {
    key: "myabtest"
    value: 0
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
    value: ""
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.06194314895751282
  }
}



#### gRPC Request

In [17]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.3612252639333452
  }
}

Response:
meta {
  puid: "jl72f5r3rkfgmntnu3d4bgf1e1"
  routing {
    key: "myabtest"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
    value: ""
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07206177651670163
  }
}



In [18]:
r = sc.predict(gateway="seldon",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.8232759661612369
  }
}

Response:
meta {
  puid: "9stsrlrtg26rgv495t0jv650d1"
  routing {
    key: "myabtest"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "myabtest"
    value: ""
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.10974087173093611
  }
}



In [19]:
!helm delete myabtest --purge

release "myabtest" deleted


## Serve Multi-Armed Bandit

In [20]:
!helm install ../../helm-charts/seldon-mab --name mymab --set oauth.key=oauth-key --set oauth.secret=oauth-secret --namespace test1

NAME:   mymab
LAST DEPLOYED: Thu May  9 14:10:59 2019
NAMESPACE: test1
STATUS: DEPLOYED

RESOURCES:
==> v1alpha2/SeldonDeployment
NAME   AGE
mymab  0s




In [21]:
!kubectl rollout status deploy/mymab-abtest-41de5b8 -n test1
!kubectl rollout status deploy/mymab-abtest-b8038b2 -n test1
!kubectl rollout status deploy/mymab-abtest-df66c5c -n test1

Waiting for deployment "mymab-abtest-41de5b8" rollout to finish: 0 of 1 updated replicas are available...
deployment "mymab-abtest-41de5b8" successfully rolled out
deployment "mymab-abtest-b8038b2" successfully rolled out
deployment "mymab-abtest-df66c5c" successfully rolled out


### Get predictions

In [22]:
sc = SeldonClient(deployment_name="mymab",namespace="test1",oauth_key="oauth-key",oauth_secret="oauth-secret")

In [23]:
r = sc.predict(gateway="seldon",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.08889991742801762
  }
}

Response:
meta {
  puid: "mtmtjcp084o8rt32d7mf6ilcrj"
  routing {
    key: "eg-router"
    value: 0
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.05584202643308697
  }
}



In [24]:
r = sc.predict(gateway="ambassador",transport="rest")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.8640733331929078
  }
}

Response:
meta {
  puid: "8t5bi7f1n5lcm9j3qa3qf6qg5l"
  routing {
    key: "eg-router"
    value: 0
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.11379060152228694
  }
}



#### gRPC Request

In [25]:
r = sc.predict(gateway="ambassador",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.774689972995667
  }
}

Response:
meta {
  puid: "ir9slmf3ajkia1em6dcrje545a"
  routing {
    key: "eg-router"
    value: 1
  }
  requestPath {
    key: "classifier-2"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.10508335441938535
  }
}



In [26]:
r = sc.predict(gateway="seldon",transport="grpc")
print(r)

Success:True message:
Request:
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.4248889584516323
  }
}

Response:
meta {
  puid: "muacde1bh7fvpmq1pljf7huics"
  routing {
    key: "eg-router"
    value: 0
  }
  requestPath {
    key: "classifier-1"
    value: "seldonio/mock_classifier:1.0"
  }
  requestPath {
    key: "eg-router"
    value: "seldonio/mab_epsilon_greedy:1.1"
  }
}
data {
  names: "proba"
  tensor {
    shape: 1
    shape: 1
    values: 0.07643660719233908
  }
}



In [27]:
!helm delete mymab --purge

release "mymab" deleted
